In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
import pickle
import numpy as np
import random
from IPython.display import display, HTML
import sys
import os
from contextlib import contextmanager


nltk.download('punkt')
nltk.download('wordnet')

# HTML and CSS used for the background color for chatbot responses
style_code = '''
<style>
    body {
        background-color: orange; /* Background color for the entire notebook */
        font-family: Arial, sans-serif;
    }
    .chatbot-container {
     padding: 20px;
        border-radius: 10px;
        box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
        margin-bottom: 20px;
    }
    .chat-row {
        display: flex;
        align-items: center;
        margin-bottom: 10px;
    }
    .bot-response {
        background-color: #add8e6; /* Light blue background for responses */
        padding: 10px;
        border-radius: 5px;
        color: black;
        display: inline-block;
        max-width: 70%;
        word-wrap: break-word;
        margin-left: auto; /* Align to the right */
    }
    .user-input {
        background-color: #d3ffd3; /* Light green background for user input */
        padding: 10px;
        border-radius: 5px;
        color: black;
        display: inline-block;
        max-width: 70%;
         word-wrap: break-word;
        margin-right: auto; /* Align to the left */
    }
</style>
'''


display(HTML(style_code))
class UniversityChatBot:
    def __init__(self):
        self.intents = {
            'course_info': [
                'Tell me about the course',
                'What is the course syllabus?',
                'Overview of the class',
                'Course subject information',
                'What courses are offered this semester?',
                'What is the syllabus for a specific course?',
                'Who is the instructor for a specific course?',
                'What are the prerequisites for a course?',
                'What is the schedule for a specific course?',
                'How can I register for a course?',
                'What is the grading policy for a course?',
                'Where can I find textbooks for my courses',
                'How can I book a study room in the library?'
            ],
            'schedule_inquiry': [
                'When does the course start?',
                'What is the end date of the course?',
                'Course schedule details',
                'Course dates',
                'Have there been any changes to my class schedule?',
                'When is the next holiday or break?',
                'When are my lab sessions?',
                'Is there a gym on campus?',
                'Where can I find mess on campus?',
                'What health services are available on campus?'
            ],
            'campus_facilities': [
                'Where is the library?',
                'Tell me about campus facilities',
                'Where can I find the gym?',
                'Information on parking',
                'How can I access the computer labs?'
                
            ],
            'deadlines': [
                'What is the deadline for the assignment?',
                'When is the assignment due?',
                'Tell me the submission dates',
                'Assignment due date',
                'What is the last day to register for exams?',
                'What is the application deadline for the graduate program?',
                'When is the last day to drop a course?',
                'When is the deadline to apply for scholarships?'
            ],
            'academic_support': [
                'Do you offer tutoring?',
                'Help with academic support',
                'Where can I get academic assistance?',
                'Tutoring center location',
                'How can I join or form a study group?',
                'How can I access tutoring services?',
                'How do I schedule an appointment with an academic advisor?',
                'Where can I get help with my writing assignments?',
                'Where can I find additional learning resources for my courses?'
                 ]}
        self.responses = {
            'course_info': "To view detailed information about your courses, including course descriptions, prerequisites, and instructor details, please check the Course Catalog on the university website.",
            'schedule_inquiry': "This will be be informed through notification or n email has been sent to your registered email address ",
            'campus_facilities': 'Our campus offers a wide range of facilities, including a state-of-the-art library, multiple computer labs, sports complexes, and dining halls. We also have on-campus medical services and student counseling.To know about please check website section campus facilities',
            'deadlines': "Students are informed that they will receive both notifications and emails to ensure they don't miss important deadlines.",
            'academic_support':"You can access academic support services like tutoring, advising, and writing assistance through the website or by visiting the relevant centers on campus. "
        }
        
        self.words = []
        self.classes = []
        self.documents = []
        self.ignore_letters = ['?', '!', '.', ',']
        self.lemmatizer = WordNetLemmatizer()

        # Create training data from intents
        for intent, patterns in self.intents.items():
            for pattern in patterns:
                word_list = nltk.word_tokenize(pattern)
                self.words.extend(word_list)
                self.documents.append((word_list, intent))
                if intent not in self.classes:
                    self.classes.append(intent)

        # Lemmatize and filter out ignore letters
        self.words = [self.lemmatizer.lemmatize(word.lower()) for word in self.words if word not in self.ignore_letters]
        self.words = sorted(set(self.words))
        self.classes = sorted(set(self.classes))

        # Save words and classes to files
        with open('words.pkl', 'wb') as f:
            pickle.dump(self.words, f)
        with open('classes.pkl', 'wb') as f:
            pickle.dump(self.classes, f)

        # Create training data
        training = []
        output_empty = [0] * len(self.classes)
        for document in self.documents:
            bag = []
            word_patterns = document[0]
            word_patterns = [self.lemmatizer.lemmatize(word.lower()) for word in word_patterns]
            for word in self.words:
                bag.append(1) if word in word_patterns else bag.append(0)
            output_row = list(output_empty)
            output_row[self.classes.index(document[1])] = 1
            training.append(bag + output_row)
        
        random.shuffle(training)
        training = np.array(training)

        # Split into features and labels
        self.trainX = training[:, :len(self.words)]
        self.trainY = training[:, len(self.words):]

        # Build and compile the model
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(128, input_shape=(len(self.trainX[0]),), activation='relu'))
        self.model.add(tf.keras.layers.Dropout(0.5))
        self.model.add(tf.keras.layers.Dense(len(self.classes), activation='softmax'))

        sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.5, nesterov=True)
        self.model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

        # Redirect stdout to suppress training logs
        with self.suppress_stdout():
            self.model.fit(self.trainX, self.trainY, epochs=200, batch_size=5, verbose=1)
        
        self.model.save('my_model.keras')

    @contextmanager
    def suppress_stdout(self):
        """ Context manager to suppress stdout """
        with open(os.devnull, 'w') as fnull:
            old_stdout = sys.stdout
            sys.stdout = fnull
            try:
                yield
            finally:
                sys.stdout = old_stdout

    def get_intent(self, user_input):
        # Tokenize and preprocess user input
        user_words = nltk.word_tokenize(user_input)
        user_words = [self.lemmatizer.lemmatize(word.lower()) for word in user_words if word not in self.ignore_letters]

        # Create a bag of words for the user input
        bag = [0] * len(self.words)
        for word in user_words:
             if word in self.words:
                bag[self.words.index(word)] = 1

        # Predict intent
        prediction = self.model.predict(np.array([bag], dtype=np.float32))[0]
        intent_index = np.argmax(prediction)
        confidence = prediction[intent_index]

        

        # Confidence threshold
        confidence_threshold = 0.7
        if confidence < confidence_threshold:
            return 'unknown'
        
        return self.classes[intent_index]

    def get_response(self, intent):
        if intent == 'unknown':
            return "Sorry, I didn't understand that. Can you please ask something else?"
        else:
            return self.responses.get(intent)

    def chat(self):
        print("Hello! I'm your university chatbot. How can I assist you today?")
        while True:
            # Collect user input
            user_input = input("You:") 

            if user_input.lower() in ['quit', 'exit', 'bye']:
                html_content = f'''
               <div class="chatbot-container">
                  <div class="chat-row">
                     <div class="bot-response"><h3> Chatbot:</h3>Goodbye! Have a great day!</div>
                  </div>
                </div>
                '''
                display(HTML(html_content))
                break
            intent = self.get_intent(user_input)
            response = self.get_response(intent)

            # Display chatbot response only
            html_content = f'''
            <div class="chatbot-container">
                <div class="chat-row">
                    <div class="bot-response"><h3> Chatbot:</h3>{response}</div>
                </div>
            </div>
            '''
            display(HTML(html_content))

# Instantiate and run the chatbot
bot = UniversityChatBot()
bot.chat()